In [25]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import torch.nn.functional as F

class Autoencoder(pl.LightningModule):
    def __init__(self, lr=1e-3):
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2, stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2, stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2, stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=2, stride=2),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=32, out_channels=3, kernel_size=2, stride=2),
            nn.ReLU()
        )
        
        self.lr = lr

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def training_step(self, batch, batch_idx):
        x, _ = batch
        x_hat = self(x)
        loss = F.mse_loss(x_hat, x)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        x_hat = self(x)
        val_loss = F.mse_loss(x_hat, x)
        self.log('val_loss', val_loss, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

# Setup dataset and dataloader
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load dataset from folder data/raw
dataset = datasets.ImageFolder(root='data/raw', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Initialize the model
torch.manual_seed(42)
model = Autoencoder()

# Setup PyTorch Lightning Trainer with checkpoints and early stopping
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints',
    filename='best-checkpoint',
    save_top_k=1,
    mode='min'
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min'
)

trainer = pl.Trainer(
    max_epochs=-1,
    callbacks=[checkpoint_callback, early_stopping_callback],
)

# Train the model
trainer.fit(model, train_loader, val_loader)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniconda/base/envs/vision/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /Users/izzal/almont-classification/checkpoints exists and is not empty.

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 1.6 M 
1 | decoder | Sequential | 697 K 
---------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.063     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniconda/base/envs/vision/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/homebrew/Caskroom/miniconda/base/envs/vision/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/homebrew/Caskroom/miniconda/base/envs/vision/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (39) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [27]:
model.eval()
X_train = model.encoder(torch.stack([x for x, _ in train_dataset]))
y_train = torch.tensor([y for _, y in train_dataset])
X_test = model.encoder(torch.stack([x for x, _ in val_dataset]))
y_test = torch.tensor([y for _, y in val_dataset])

X_train = [x.detach().numpy().flatten() for x in X_train]
X_test = [x.detach().numpy().flatten() for x in X_test]
y_train = y_train.detach().numpy()
y_test = y_test.detach().numpy()

In [28]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

svc = SVC()
params = {
    'C': np.logspace(-3, 3, 7),
    'gamma': np.logspace(-3, 3, 7),
}

rscv = RandomizedSearchCV(
    svc,
    params,
    cv=10,
    n_jobs=-1,
    verbose=3,
    random_state=42,
    scoring='accuracy',
)

In [29]:
rscv.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 2/10] END .............C=0.01, gamma=1000.0;, score=0.304 total time=  42.6s
[CV 3/10] END .............C=0.01, gamma=1000.0;, score=0.304 total time=  43.2s
[CV 6/10] END .............C=0.01, gamma=1000.0;, score=0.306 total time=  41.6s
[CV 4/10] END .............C=0.01, gamma=1000.0;, score=0.304 total time=  43.4s
[CV 5/10] END .............C=0.01, gamma=1000.0;, score=0.306 total time=  42.9s
[CV 1/10] END .............C=0.01, gamma=1000.0;, score=0.304 total time=  43.8s
[CV 7/10] END .............C=0.01, gamma=1000.0;, score=0.306 total time=  42.1s
[CV 8/10] END .............C=0.01, gamma=1000.0;, score=0.306 total time=  42.3s
[CV 9/10] END .............C=0.01, gamma=1000.0;, score=0.306 total time=  40.8s
[CV 10/10] END ............C=0.01, gamma=1000.0;, score=0.306 total time=  42.6s
[CV 1/10] END ..............C=1000.0, gamma=1.0;, score=0.304 total time=  42.4s
[CV 2/10] END ..............C=1000.0, gamma=1.

RandomizedSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
                   param_distributions={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                                        'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
                   random_state=42, scoring='accuracy', verbose=3)

In [24]:
rscv.best_score_

0.7073935483870968